In [1]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


**[문제 1: 특정 장르의 책 찾기]**

- **문제 설명**:
데이터베이스에 새로운 필드로 **`genre`**를 책 데이터에 추가하였습니다. 사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
"fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [ ]:
def find_genre(db,genre):
    collection = db.books
    query = {"genre": genre}
    projection = {"_id": 0, "title": 1, "author": 1}

    books = collection.find(query, projection)
    for book in books:
        print(book)

client = MongoClient('mongodb://localhost:27017/')
find_genre(client.local,'fantasy')

**[문제 2: 감독별 평균 영화 평점 계산]**

- **문제 설명**:
각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**:
모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [21]:
def avg_rating(db,director):
    collection = db.movies
    query = {"director": director}
    projection = {"_id": 0, "rating": 1}
    ratings = collection.find(query, projection)
    r=[]
    for rating in ratings:
        r.append(rating['rating'])
    print(sum(r)/len(r))
client = MongoClient('mongodb://localhost:27017/')
avg_rating(client.local,'Christopher Nolan')

8.8
8.6
9.0
8.799999999999999


**[문제 3: 특정 사용자의 최근 행동 조회]**

- **문제 설명**:
특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [42]:
def find_action_log(db,user_id):
    collection = db.user_actions
    query = {"user_id": user_id}
    sort = [("timestamp", -1)]

    actions = collection.find(query).sort(sort).limit(5)
    for action in actions:
        print(action)

client = MongoClient('mongodb://localhost:27017/')
find_action_log(client.local,1)


{'_id': ObjectId('6628b89a085f209d9af7bbdf'), 'user_id': 1, 'action': 'view', 'timestamp': '2023-04-12T09:00:00Z'}
{'_id': ObjectId('6628b89a085f209d9af7bbde'), 'user_id': 1, 'action': 'click', 'timestamp': '2023-04-12T08:00:00Z'}


**[문제 4: 출판 연도별 책의 수 계산]**

- **문제 설명** :
데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [51]:
def count_books(db):
    collection = db.books
    pipeline = [
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]

    books = collection.aggregate(pipeline)
    for book in books:
        print(book)

client = MongoClient('mongodb://localhost:27017/')
count_books(client.local)

{'_id': 1987, 'count': 1}
{'_id': 2009, 'count': 1}
{'_id': 2002, 'count': 1}


**[문제 5: 특정 사용자의 행동 유형 업데이트]**

- **문제 설명**:
특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.

In [65]:
from datetime import datetime
def update_action(db,user_id, date, old_action, new_action):
    collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}

    result = collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

client = MongoClient('mongodb://localhost:27017/')
update_action(client.local, 1, datetime(2024, 4, 15), "view", "seen")

Updated 0 documents.
